In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

from custom_estimator import Estimator
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import os
import glob
from sklearn.externals import joblib
from encoding import FreqeuncyEncoding
import warnings
warnings.filterwarnings("ignore")
from sklearn.externals import joblib

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
train=pd.read_pickle('../train_v2.pkl')
test=pd.read_pickle('../test_v2.pkl')

In [3]:
feat1=pd.read_csv('../feats_user_coup_item_no_lag_v2.csv')
feat2=pd.read_csv('../coupon_item_feats_no_lag_v2.csv')
feat3=pd.read_csv('../brand_cust_feats_no_lag_v2.csv')
feat4=pd.read_csv('../brand_feats_no_lag_v2.csv')
feat6=pd.read_csv('../coupon_mean_encoding_v2.csv')
feat7=pd.read_csv('../camp_cust_feats_v2.csv')
feat8=pd.read_csv('../cust_coup_hist3_v2.csv')
feat9=pd.read_csv('../customer_coupon_var1_v2.csv')
feat10=pd.read_csv('../customer_coupon_var2_v2.csv')
feat11=pd.read_csv('../coup_hist1_v2.csv')
feat12=pd.read_csv('../test_feat_2.csv')
feat13=pd.read_csv('../test_feat.csv')

In [4]:
drop_cols=['coupon_discount_mean', 'selling_price_mean', 'other_discount_sum', 'other_discount_mean', 'cd_sp_ratio_mean', 
 'cd_sp_ratio_sum', 'coupon_discount_mean_coup', 'selling_price_mean_coup', 'other_discount_sum_coup',
 'other_discount_mean_coup', 'cd_sp_ratio_mean_coup', 'cd_sp_ratio_sum_coup', 'selling_price_sum_brand_cust',
 'coupon_discount_mean_brand_cust', 'selling_price_mean_brand_cust', 'other_discount_sum_brand_cust', 
 'other_discount_mean_brand_cust', 'cd_sp_ratio_mean_brand_cust', 'cd_sp_ratio_sum_brand_cust', 'selling_price_sum_brand', 
 'coupon_discount_mean_brand', 'selling_price_mean_brand', 'other_discount_sum_brand', 'other_discount_mean_brand', 
 'cd_sp_ratio_mean_brand', 'cd_sp_ratio_sum_brand', 'selling_price_sum_camp_cust', 'coupon_discount_mean_camp_cust', 
 'selling_price_mean_camp_cust', 'other_discount_sum_camp_cust', 'other_discount_mean_camp_cust',
 'cd_sp_ratio_mean_camp_cust', 'cd_sp_ratio_sum_camp_cust', 'selling_price_sum_cust_var3', 'coupon_discount_mean_cust_var3',
 'selling_price_mean_cust_var3', 'other_discount_sum_cust_var3', 'other_discount_mean_cust_var3', 
 'cd_sp_ratio_mean_cust_var3', 'cd_sp_ratio_sum_cust_var3', 'selling_price_sum_cust_var1', 'coupon_discount_mean_cust_var1',
 'selling_price_mean_cust_var1', 'other_discount_sum_cust_var1', 'other_discount_mean_cust_var1', 'cd_sp_ratio_mean_cust_var1',
 'cd_sp_ratio_sum_cust_var1', 'selling_price_sum_cust_var2', 'coupon_discount_mean_cust_var2', 'selling_price_mean_cust_var2',
 'other_discount_sum_cust_var2', 'other_discount_mean_cust_var2', 'cd_sp_ratio_mean_cust_var2', 'cd_sp_ratio_sum_cust_var2', 
 'selling_price_sum_coup_var1', 'coupon_discount_mean_coup_var1', 'selling_price_mean_coup_var1', 
 'other_discount_sum_coup_var1', 'other_discount_mean_coup_var1', 'cd_sp_ratio_mean_coup_var1', 'cd_sp_ratio_sum_coup_var1']
len(drop_cols)

61

In [5]:
train=train.merge(feat1,on='id')
test=test.merge(feat1,on='id')
train=train.merge(feat2,on='id')
test=test.merge(feat2,on='id')
train=train.merge(feat3,on='id')
test=test.merge(feat3,on='id')
train=train.merge(feat4,on='id')
test=test.merge(feat4,on='id')
train=train.merge(feat6,on='coupon_id')
test=test.merge(feat6,on='coupon_id')
train=train.merge(feat7,on=['campaign_id','customer_id'])
test=test.merge(feat7,on=['campaign_id','customer_id'])
train=train.merge(feat8,on=['campaign_id','customer_id'])
test=test.merge(feat8,on=['campaign_id','customer_id'])
train=train.merge(feat9,on=['campaign_id','customer_id'])
test=test.merge(feat9,on=['campaign_id','customer_id'])
train=train.merge(feat10,on=['campaign_id','customer_id'])
test=test.merge(feat10,on=['campaign_id','customer_id'])
train=train.merge(feat11,on=['campaign_id','coupon_id'])
test=test.merge(feat11,on=['campaign_id','coupon_id'])
train=train.merge(feat12,on='id')
test=test.merge(feat12,on='id')
train=train.merge(feat13,on='id')
test=test.merge(feat13,on='id')


In [6]:
train.drop(drop_cols,axis=1,inplace=True)
test.drop(drop_cols,axis=1,inplace=True)

In [7]:
train.shape

(78369, 133)

In [8]:
target=train[['id','redemption_status']]
train.drop('redemption_status',axis=1,inplace=True)
df=pd.concat((train,test),axis=0)

In [9]:
df['cat1']=(df['customer_id'].astype('str')+'_'+df['campaign_id'].astype('str'))

In [10]:
df.shape

(128595, 133)

In [11]:
df['num_coupons_per_camp']=df['campaign_id'].map(df.groupby('campaign_id')['coupon_id'].nunique().to_dict())
df['num_cust_per_camp']=df['campaign_id'].map(df.groupby('campaign_id')['customer_id'].nunique().to_dict())
df['num_cust_per_coup']=df['coupon_id'].map(df.groupby('coupon_id')['customer_id'].nunique().to_dict())


In [12]:
df.head()

,id,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,camp_duration,campaign_start_day,campaign_start_dow,campaign_end_day,campaign_end_dow,days_since_last_campaign_start,days_since_last_campaign_end,days_between_start_end_of_prev_campaign,trans_shape,unique_item,unique_brand,unique_category,item_id,qty_mean,qty_sum,qty_std,qty_max,sp_mean,sp_sum,sp_std,sp_max,od_mean,od_sum,od_std,od_max,coupon_redeemed_mean,coupon_redeemed_sum,coupon_redeemed_std,coupon_redeemed_max,cd_mean,cd_sum,cd_std,cd_max,num_coupons_per_brand_mean,num_coupons_per_brand_sum,num_coupons_per_brand_std,num_coupons_per_brand_max,num_coupons_per_category_mean,num_coupons_per_category_sum,num_coupons_per_category_std,num_coupons_per_category_max,num_coupons_per_item_mean,num_coupons_per_item_sum,num_coupons_per_item_std,num_coupons_per_item_max,cost_price_mean,cost_price_sum,cost_price_std,cost_price_max,selling_price_ewm_mean,selling_price_ewm_sum,selling_price_ewm_std,selling_price_ewm_max,other_discount_ewm_mean,other_discount_ewm_sum,other_discount_ewm_std,other_discount_ewm_max,coupon_discount_ewm_mean,coupon_discount_ewm_sum,coupon_discount_ewm_std,coupon_discount_ewm_max,price_after_OD_ewm_mean,price_after_OD_ewm_sum,price_after_OD_ewm_std,price_after_OD_ewm_max,cost_price_ewm_mean,cost_price_ewm_sum,cost_price_ewm_std,cost_price_ewm_max,coupon_redeemed_ewm_mean,coupon_redeemed_ewm_sum,coupon_redeemed_ewm_std,coupon_redeemed_ewm_max,age_range,marital_status,rented,family_size,no_of_children,income_bracket,num_bought,num_discount_availed,frac_discount_availed,sum_discount,selling_price_sum,num_bought_coup,num_discount_availed_coup,frac_discount_availed_coup,sum_discount_coup,selling_price_sum_coup,num_bought_brand_cust,num_discount_availed_brand_cust,frac_discount_availed_brand_cust,sum_discount_brand_cust,num_bought_brand,num_discount_availed_brand,frac_discount_availed_brand,sum_discount_brand,tot_redemption,mean_redemption,num_bought_camp_cust,num_discount_availed_camp_cust,frac_discount_availed_camp_cust,sum_discount_camp_cust,num_bought_cust_var3,num_discount_availed_cust_var3,frac_discount_availed_cust_var3,sum_discount_cust_var3,num_bought_cust_var1,num_discount_availed_cust_var1,frac_discount_availed_cust_var1,sum_discount_cust_var1,num_bought_cust_var2,num_discount_availed_cust_var2,frac_discount_availed_cust_var2,sum_discount_cust_var2,num_bought_coup_var1,num_discount_availed_coup_var1,frac_discount_availed_coup_var1,sum_discount_coup_var1,test1_var2,test2_var2,test3_var2,test1,test2,test3,cat1,num_coupons_per_camp,num_cust_per_camp,num_cust_per_coup
0,1,13,27,1053,X,2013-05-19,2013-07-05,47,19,6,5,4,27.0,42.0,-5.0,270.0,183.0,73.0,8.0,14458.0,297.185185,80240.0,1491.301197,12444.0,169.459593,45754.09,405.939171,5164.54,-29.522222,-7971.00,81.813360,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,138.796296,37475.0,146.143432,297.0,567.511111,153228.0,327.940501,776.0,1.466667,396.0,1.826285,10.0,198.981815,53725.09,454.546430,6055.04,169.899428,45872.845461,179.226537,1590.382953,-29.592802,-7990.056492,36.658015,-0.264829,0.000000,0.000000,0.000000,0.0,199.492229,53862.901953,201.774242,1854.962017,199.492229,53862.901953,201.774242,1854.962017,0.000000,0.000000,0.000000,0.000000,46-55,NaN,0.0,1,NaN,5.0,0.0,0.0,0.0,0.0,0.00,3309.0,158.0,0.047749,-4662.46,286278.21,0.0,0.0,0.000000,0.00,5614.0,228.0,0.040613,-6879.73,21,0.083665,32.0,0.0,0.000000,0.00,32.0,0.0,0.000000,0.00,33.0,0.0,0.000000,0.00,217684.0,5062.0,0.023254,-192327.19,550.0,55.0,0.1,-2265.3,65.0,627.0,-2468.34,191.0,3713.0,-5349.93,1053_13,207,1077,122
1,370,13,27,1168,X,2013-05-19,2013-07-05,47,19,6,5,4,27.0,42.0,-5.0,778.0,589.0,208.0,11.0,45069.0,31.646530,24621.0,601.380970,12707.0,84.315694,65597.61,83.211634,1244.21,-20.941401,-16292.41,34.681668,0.0,0.044987,35.0,0.207409,1.0,-1.442185,-1122.02,8.204593,0.0,89.429306,69576.0,131.362901,297.0,579.595116,450925.0,307.268586,776.0,1.916452,1491.0,1.755876,10.0,106.699280,83012.04,100.610291,1289.

In [13]:
cat_cols=['campaign_id','coupon_id','customer_id','campaign_type','campaign_start_day','campaign_start_dow',
         'campaign_end_day','campaign_end_dow','item_id',
          'age_range','marital_status','family_size','no_of_children',
          'cat1'
         ]
id_cols=['id']
date_cols=['start_date','end_date']

In [14]:
from sklearn.model_selection import GroupKFold
folds=GroupKFold(n_splits=5)
temp=df[df.id.isin(train.id)]
folds1=[(x,y) for x,y in  folds.split(temp,temp.merge(target,on='id').redemption_status,temp.coupon_id)]
folds2=[(x,y) for x,y in  folds.split(temp,temp.merge(target,on='id').redemption_status,temp.campaign_id)]

In [15]:
fe=FreqeuncyEncoding(normalize=1,return_df=True,categorical_columns=cat_cols)
df=fe.fit_transform(df)

In [16]:
use_cols=df.columns[~df.columns.isin(date_cols+id_cols)].tolist()

In [17]:
train_df=df[df.id.isin(train.id)]
test_df=df[df.id.isin(test.id)]
train_df.shape,test_df.shape

((78369, 136), (50226, 136))

In [18]:
train_df=train_df.merge(target,on='id')

In [19]:
params2={'boosting_type': 'gbdt',
 'colsample_bytree': 0.7000000000000001,
 'learning_rate': 0.1,
 'metric': 'None',
 'min_child_weight': 30.0,
 'n_estimators': 5000,
 'n_jobs': -1,
 'num_leaves': 64,
 'objective': 'binary',
 'subsample': 0.8,
 'subsample_freq': 5}
est_lgb=Estimator(model=LGBMClassifier(**params2),n_jobs=-1,early_stopping_rounds=300,
              validation_scheme=folds2
                 )
xgb_params={'colsample_bytree': 0.4,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 30.0,
 'n_estimators': 5000,
 'objective': 'binary:logistic',
 'subsample': 1.0}
est_xgb=Estimator(model=XGBClassifier(**xgb_params),n_jobs=-1,early_stopping_rounds=100,
              validation_scheme=folds2
                 )


In [20]:
oof_lgb=est_lgb.fit_transform(train_df[use_cols].values,train_df['redemption_status'].values)
est_lgb.avg_cv_score

Training until validation scores don't improve for 300 rounds
[100]	valid_0's auc: 0.949647	valid_1's auc: 0.973438
[200]	valid_0's auc: 0.94632	valid_1's auc: 0.983697
[300]	valid_0's auc: 0.946159	valid_1's auc: 0.989999
Early stopping, best iteration is:
[76]	valid_0's auc: 0.950329	valid_1's auc: 0.968563
Training until validation scores don't improve for 300 rounds
[100]	valid_0's auc: 0.958943	valid_1's auc: 0.976487
[200]	valid_0's auc: 0.954735	valid_1's auc: 0.986938
[300]	valid_0's auc: 0.95086	valid_1's auc: 0.992037
Early stopping, best iteration is:
[62]	valid_0's auc: 0.961868	valid_1's auc: 0.968848
Training until validation scores don't improve for 300 rounds
[100]	valid_0's auc: 0.885065	valid_1's auc: 0.984109
[200]	valid_0's auc: 0.8722	valid_1's auc: 0.991057
[300]	valid_0's auc: 0.859366	valid_1's auc: 0.994458
[400]	valid_0's auc: 0.85165	valid_1's auc: 0.996416
Early stopping, best iteration is:
[116]	valid_0's auc: 0.887356	valid_1's auc: 0.985949
Training until

0.9274144201149053

In [21]:
est_lgb.feature_importance_df(train_df[use_cols].columns)

,column,feature_importance,rank
87,selling_price_sum,0.055977,1
126,test1,0.029076,2
102,mean_redemption,0.025717,3
103,num_bought_camp_cust,0.021911,4
99,frac_discount_availed_brand,0.021554,5
97,num_bought_brand,0.021431,6
113,frac_discount_availed_cust_var1,0.020872,7
125,test3_var2,0.020482,8
93,num_bought_brand_cust,0.018126,9
2,customer_id,0.017866,10


In [22]:
oof_xgb=est_xgb.fit_transform(train_df[use_cols].values,train_df['redemption_status'].values)
est_xgb.avg_cv_score

[0]	validation_0-auc:0.801562
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.945644
[200]	validation_0-auc:0.947067
Stopping. Best iteration:
[181]	validation_0-auc:0.94804

[0]	validation_0-auc:0.856742
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.95926
Stopping. Best iteration:
[92]	validation_0-auc:0.960747

[0]	validation_0-auc:0.723064
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.865439
Stopping. Best iteration:
[39]	validation_0-auc:0.878512

[0]	validation_0-auc:0.695969
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.905941
[200]	validation_0-auc:0.923925
[300]	validation_0-auc:0.935399
[400]	validation_0-auc:0.936082
[500]	validation_0-auc:0.936421
Stopping. Best iteration:
[487]	validation_0-auc:0.937604

[0]	validation_0-auc:0.757743
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]

0.9283476282628568

In [23]:
est_xgb.feature_importance_df(train_df[use_cols].columns)

,column,feature_importance,rank
84,num_discount_availed,0.078989,1
86,sum_discount,0.057594,2
85,frac_discount_availed,0.042984,3
83,num_bought,0.039719,4
104,num_discount_availed_camp_cust,0.031849,5
87,selling_price_sum,0.031536,6
108,num_discount_availed_cust_var3,0.021534,7
112,num_discount_availed_cust_var1,0.017561,8
3,campaign_type,0.016668,9
7,campaign_end_day,0.015059,10


In [24]:
sub=pd.read_csv('../sample_submission_Byiv0dS.csv')

In [25]:
preds_lgb=est_lgb.transform(test_df[use_cols].values)
preds_xgb=est_xgb.transform(test_df[use_cols].values)

In [26]:
pd.DataFrame({'id':train_df['id'],'redemption_status': oof_xgb}).to_csv('OOF_baseline_xgb_grp_k_camp_more_feats.csv',index=False)
pd.DataFrame({'id':train_df['id'],'redemption_status': oof_lgb}).to_csv('OOF_baseline_lgb_grp_k_camp_more_feats.csv',index=False)

In [27]:
pd.DataFrame({'id':test_df['id'],'redemption_status': preds_xgb}).to_csv('baseline_xgb_grp_k_camp_more_feats.csv',index=False)
pd.DataFrame({'id':test_df['id'],'redemption_status': preds_lgb}).to_csv('baseline_lgb_grp_k_camp_more_feats.csv',index=False)